<a href="https://colab.research.google.com/github/Justinhoejj/nusfintech_ml_training_project2/blob/main/NUS_Fintech_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving stock_data.csv to stock_data.csv


In [ ]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import io
import pandas as pd

df = pd.read_csv(io.BytesIO(uploaded['stock_data.csv']))
pos_df = df[df['Sentiment'] == 1]
neg_df = df[df['Sentiment'] == -1]

In [ ]:
print("Number of negative sentiments")
neg_df.count()

Number of negative sentiments


Text         2106
Sentiment    2106
dtype: int64

In [ ]:
print("Number of positive sentiments")
pos_df.count()

Number of positive sentiments


Text         3685
Sentiment    3685
dtype: int64

In [ ]:
TRAIN_SIZE = 1500
TEST_SIZE = 600

In [ ]:
pos_train, pos_test = pos_df[:TRAIN_SIZE], pos_df[TRAIN_SIZE: TRAIN_SIZE+TEST_SIZE]
neg_train, neg_test = neg_df[:TRAIN_SIZE], neg_df[TRAIN_SIZE: TRAIN_SIZE+TEST_SIZE]

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.2 MB/s 
     |████████████████████████████████| 182 kB 71.7 MB/s 
     |████████████████████████████████| 7.6 MB 64.3 MB/s 


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

In [ ]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

In [ ]:
nlp = pipeline('sentiment-analysis', model=finbert, tokenizer=tokenizer)

In [ ]:
draft_df = pos_df.head(2106).copy()
draft_df = draft_df.append(neg_df)
results = nlp(draft_df['Text'].tolist())

In [ ]:
labels = list(map(lambda x : 0 if x['label'] == "Neutral" else (1 if x['label'] == "Positive" else -1), results))
draft_df['Prediction'] = labels

## Analysis on positive sentiments

Precision is quality of positive prediction made by model.




In [ ]:
true_positives = draft_df[(draft_df['Sentiment'] == 1) & (draft_df['Prediction'] == 1)]

In [ ]:
precision = true_positives.count() / draft_df[draft_df['Prediction'] == 1].count()
print("Precision: ", precision)

Precision:  Text          0.807245
Sentiment     0.807245
Prediction    0.807245
dtype: float64


Recall is the models ability to detect true positives

In [ ]:
false_negatives = draft_df[(draft_df['Sentiment'] == 1) & (draft_df['Prediction'] != 1)]
recall = true_positives.count() / (true_positives.count() + false_negatives.count())
print("Recall:", recall)

Recall: Text          0.296296
Sentiment     0.296296
Prediction    0.296296
dtype: float64


## Analysis on negative sentiments

Precision is quality of positive prediction made by model.

In [ ]:
true_positives = draft_df[(draft_df['Sentiment'] == -1) & (draft_df['Prediction'] == -1)]
precision = true_positives.count() / draft_df[draft_df['Prediction'] == -1].count()
print("Precision: ", precision)

Precision:  Text          0.821429
Sentiment     0.821429
Prediction    0.821429
dtype: float64


Recall is the models ability to detect true positives

In [ ]:
false_negatives = draft_df[(draft_df['Sentiment'] == -1) & (draft_df['Prediction'] != -1)]
recall = true_positives.count() / (true_positives.count() + false_negatives.count())
print("Recall:", recall)

Recall: Text          0.207502
Sentiment     0.207502
Prediction    0.207502
dtype: float64


## Conclusion

Low recall and high precision for both positive and negative sentiments.

In [ ]:
accuracy = draft_df[(draft_df['Sentiment'] == draft_df['Prediction'])].count()
print('Accuracy:', accuracy[0]/draft_df.count()[0])

Accuracy: 0.25189933523266855
